In [ ]:
import os

base_dir = '/home/ubuntu/data_value/bigdata-transportation'

ids = []

for root, dirs, files in os.walk(base_dir):

    for f in files:

        i = f.split('_')[1].split('.')[0]

        ids.append(i)
        print(i)



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import os
import traceback

base_dir = '/home/ubuntu/data_value/bigdata-transportation-html'

os.makedirs(base_dir, exist_ok=True)

for i in ids:

    try:
        
        url = f"https://www.bigdata-transportation.kr/frn/prdt/detail?prdtId=PRDTNUM_0000000{i}"

        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')

        driver = webdriver.Chrome(options=options)
        driver.get(url)    

        html_source = driver.page_source

        time.sleep(3)

        file_path = os.path.join(base_dir, f'{i}.html')

        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(html_source)


    except Exception as e:
        print(f'Error on file {i}')
        traceback.print_exc()


In [ ]:
import pandas as pd

df = pd.read_csv('/home/ubuntu/data_value/KDX+국가교통데이터오픈마켓+해양수산_20250729_2_category.csv')

df

In [ ]:
df.loc[df['source'] == 'bigdata-transportation']

In [ ]:
import os
from bs4 import BeautifulSoup

html_dir = '/home/ubuntu/data_value/bigdata-transportation-html' 
results = []

for filename in os.listdir(html_dir):
    if not filename.endswith('.html'):
        continue

    file_path = os.path.join(html_dir, filename)

    with open(file_path, encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    # 1. 카테고리 추출
    category_tag = soup.select_one("span.categoty-tag") or soup.select_one("span.categoty-txt")
    category = category_tag.text.strip() if category_tag else None

    # 2. 확장자 추출
    format_td = soup.find("td", id="dt-format")
    extensions = format_td.text.strip() if format_td else None

    # 3. 회사명 추출
    provider_td = soup.find("td", id="prdt-bsns-nm1")
    company = provider_td.text.strip() if provider_td else None

    print("카테고리:", category)
    print("확장자:", extensions)
    print("제공자:", company)

    file_id = filename.split('.')[0]

    
    df.loc[df['filename'] == f'definition_{file_id}.xlsx', 'category'] = category
    df.loc[df['filename'] == f'definition_{file_id}.xlsx', 'extension'] = extensions
    df.loc[df['filename'] == f'definition_{file_id}.xlsx', 'company'] = company


In [ ]:
df.to_csv("/home/ubuntu/data_value/KDX+국가교통데이터오픈마켓+해양수산_20250729_3_category.csv", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('/home/ubuntu/data_value/KDX+국가교통데이터오픈마켓+해양수산_20250730_3.csv')

df

In [ ]:
import os
from bs4 import BeautifulSoup

def extract_tags_from_type3_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    tags = []

    for span in soup.select('div.keyword-group span.keyword'):
        text = span.get_text(strip=True)
        if text.startswith('#'):
            tags.append(text.strip('# \n'))

    return tags

def traverse_directory_and_parse_tags_type3(root_dir):
    tag_results = {}  # filename -> list of tags

    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith('.html'):
                full_path = os.path.join(dirpath, filename)
                try:
                    with open(full_path, 'r', encoding='utf-8') as f:
                        html = f.read()
                        tags = extract_tags_from_type3_html(html)

                        file_id = filename.split('.')[0]    
                        df.loc[df['filename'] == f'definition_{file_id}.xlsx', 'tags'] = str(tags)

                        print(tags)
                        
                except Exception as e:
                    print(f"Failed to process {filename}: {e}")

traverse_directory_and_parse_tags_type3('/home/ubuntu/data_value/bigdata-transportation-html')

In [ ]:
df.drop('tags_list', axis='columns', inplace=True)
df.to_csv('/home/ubuntu/data_value/KDX+국가교통데이터오픈마켓+해양수산_20250730_6.csv', index=False)

In [ ]:
df = pd.read_csv('/home/ubuntu/data_value/KDX+국가교통데이터오픈마켓+해양수산_20250730_4.csv')

In [ ]:
import pandas as pd
import ast
df['tags_list'] = df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])


In [ ]:
from collections import Counter


all_tags = [tag for tags in df['tags_list'] for tag in tags]

tag_counts = Counter(all_tags)

print(tag_counts.most_common(1000))


In [ ]:
tags = [('분석', 4652), ('미디어', 4595), ('유동인구', 4559), ('뉴스', 4557), ('일별', 4554), ('위치데이터', 4517), ('빅데이터', 4516), ('통신데이터', 4507), ('방문인구', 4506), ('직장인구', 4505), ('서비스인구', 4505), ('주거인구', 4504), ('바이브컴퍼니', 4445), ('바이브', 4445), ('블로그', 4445), ('키워드', 4395), ('소셜', 4359), ('다음소프트', 4357), ('트위터', 4354), ('인스타그램', 4316), ('SNS', 4301), ('NH농협카드', 2258), ('소비', 2177), ('연관어', 2146), ('유통', 2007), ('카드', 1965), ('2021년', 1938), ('2022년', 1898), ('전국', 1605), ('KDX', 1579), ('카테고리', 1450), ('업종', 1412), ('유통소비', 1360), ('KCB', 1278), ('소비트렌드', 1232), ('성별', 1231), ('코리아크레딧뷰로', 1221), ('연관키워드', 1160), ('상품분류', 1127), ('일자별', 1123), ('e커머스', 1119), ('원문정보', 1107), ('온라인상품', 1088), ('문서', 1085), ('빈도수', 1084), ('버즈추이', 1084), ('내용', 1083), ('제목', 1082), ('원문매핑정보', 1080), ('상품현황', 1077), ('빌트온', 1071), ('URL', 1065), ('감성', 1062), ('감성연관어', 1062), ('kdx', 958), ('', 916), ('시군구', 914), ('신용정보', 898), ('가전제품', 856), ('이커머스', 840), ('시도', 815), ('가격정보', 766), ('gs25', 762), ('2020년', 755), ('GS25', 754), ('지에스', 732), ('GS', 721), ('연령별', 689), ('인기순위', 681), ('직업군별', 680), ('가격현황', 645), ('음식점', 627), ('매출', 615), ('유통데이터', 572), ('소비분석', 563), ('시장분석', 551), ('커머스', 550), ('상품유통', 546), ('소비트랜드', 546), ('씨제이올리브네트웍스', 546), ('식당', 540), ('11월', 538), ('10월', 533), ('상권', 523), ('9월', 521), ('행정동', 518), ('소비형태', 518), ('요일', 510), ('12월', 508), ('아파트', 503), ('연령대', 493), ('지역', 491), ('패션트렌드', 490), ('백화점', 475), ('매출구성비', 468), ('부동산', 456), ('쇼핑몰', 449), ('대형마트', 440), ('패션빅데이터', 427), ('지역별', 419), ('시간대', 419), ('리뷰현황', 407), ('구매후기', 407), ('텍스트마이닝', 407), ('후기정보', 406), ('판매점유율', 405), ('등록순위', 398), ('택배', 391), ('온라인쇼핑', 390), ('상품명', 390), ('송장', 388), ('주문수', 388), ('7월', 386), ('8월', 381), ('1월', 369), ('6월', 368), ('5월', 364), ('속성', 359), ('아이템트렌드', 356), ('4월', 352), ('2월', 352), ('3월', 352), ('식권', 350), ('건강기능식품', 348), ('최저가', 348), ('건기식', 346), ('의약외품', 344), ('로플랫', 344), ('오프라인', 343), ('매일경제', 342), ('소득', 340), ('loplat', 339), ('금융', 330), ('점심', 330), ('mbn', 328), ('매장방문', 326), ('맛집', 325), ('visit', 324), ('매장', 324), ('건강', 323), ('약국', 321), ('식대', 320), ('헬스케어', 318), ('kcb', 316), ('신용카드', 305), ('직장인', 293), ('경제', 285), ('컬러트렌드', 285), ('사용시간대별', 283), ('소재트렌드', 278), ('프린트트렌드', 273), ('매경', 272), ('statistics', 270), ('방문통계', 270), ('매일방송', 270), ('트렌드', 268), ('mk', 265), ('모바일식권', 262), ('공간정보', 256), ('환경정보', 256), ('주소', 256), ('뷰티', 256), ('방송', 256), ('베스트100', 253), ('일반시설물', 252), ('비씨티원', 252), ('전화번호', 250), ('베스트셀링', 249), ('매일', 249), ('보도', 244), ('의약품', 240), ('농협', 239), ('디테일트렌드', 239), ('은행', 237), ('패션', 234), ('웹로그', 232), ('언론사', 232), ('대출', 231), ('최저가격', 223), ('트렌드랭킹', 222), ('온누리에이치엔씨', 219), ('온누리hnc', 215), ('연체', 212), ('기획조사', 211), ('인테리어', 207), ('가구', 203), ('저축은행', 202), ('금융소비자', 199), ('증권사', 199), ('보험사', 199), ('KFSS', 198), ('종합뉴스', 193), ('방문자분석', 193), ('패션트렌드분석', 189), ('allseadata', 188), ('판매데이터', 182), ('구매데이터', 181), ('인공지능', 177), ('자산평가', 177), ('레저', 176), ('상의', 175), ('팬츠', 175), ('원피스', 175), ('브랜드', 175), ('데이터', 173), ('예금여력', 173), ('행동패턴분석', 173), ('소비데이터', 172), ('좌표', 168), ('고객구성비', 167), ('카드소비', 164), ('식자재', 164), ('쇼핑데이터', 163), ('온라인소비', 162), ('소득구간', 161), ('월소득', 160), ('poi', 157), ('지역화폐', 157), ('판매', 150), ('의류', 150), ('메뉴', 147), ('잡화', 142), ('매체사', 142), ('의료', 141), ('생활가전', 141), ('category', 140), ('리뷰', 139), ('편의점', 139), ('전자상거래', 136), ('요일별', 136), ('가맹점', 135), ('area', 135), ('소비동향', 132), ('온라인몰', 132), ('기업', 130), ('판매내역', 129), ('판매순위', 129), ('디지털', 129), ('쇼핑트렌드', 128), ('식품', 127), ('업력', 127), ('제조사', 126), ('비대면', 126), ('온누리', 126), ('면세점', 126), ('상품정보', 125), ('가격', 124), ('실거래가', 124), ('장소명', 123), ('홈쇼핑', 123), ('핫플레이스', 123), ('주방가전', 121), ('추세', 120), ('배달앱', 119), ('생활용품', 118), ('차량용품', 118), ('키즈', 114), ('여성', 114), ('의약', 114), ('욕실용품', 113), ('문구', 113), ('시장', 113), ('소매점', 113), ('음향가전', 112), ('하이엔드대상자비율', 112), ('상품데이터', 112), ('반려', 112), ('해외구매', 112), ('유통관련기타', 112), ('구매횟수', 112), ('취미', 112), ('식음', 112), ('드럭스토어', 112), ('사업체수', 112), ('악세사리', 112), ('도서/문구', 112), ('주방용품', 112), ('아울렛', 112), ('영상가전', 112), ('남성', 111), ('기타소비품목', 111), ('소비가능금액', 111), ('일반의약품', 111), ('otc', 110), ('올시데이터', 107), ('스커트아우터', 105), ('일반약품', 105), ('기업정보', 105), ('소비행태', 104), ('체크카드', 102), ('API', 102), ('위경도', 102), ('전자식권', 102), ('자동차', 101), ('온라인판매내역', 99), ('통계', 98), ('패션빅데이터분석', 97), ('온누리스토어', 96), ('뷰티제품', 96), ('소비자', 96), ('긍부정추이', 96), ('머신러닝', 95), ('화장품', 94), ('NLP', 93), ('스포츠', 92), ('스커트', 92), ('아우터', 91), ('추정데이터', 89), ('메뉴정보', 87), ('전국음식점', 86), ('데이블', 85), ('음식', 84), ('매장정보', 84), ('가격비교', 84), ('미용', 83), ('의류품목별판매', 82), ('ai', 81), ('시장동향', 81), ('생산정보', 78), ('청소기', 78), ('입주', 76), ('업종카테고리', 75), ('학습', 74), ('소분류시군구', 74), ('sns', 73), ('아이템', 73), ('선박', 72), ('원천', 72), ('지출', 72), ('냉장고', 71), ('서울시', 68), ('연령대별', 68), ('온누리약국', 68), ('자연어처리', 67), ('형태소', 67), ('생활', 66), ('푸드', 65), ('뉴스데이터', 65), ('기사데이터', 65), ('뉴스분석', 65), ('기사요약', 65), ('머신', 64), ('2024', 63), ('컨텐츠', 63), ('매거진', 63), ('외식트렌드', 63), ('아파트실거래가', 62), ('하이엔드', 62), ('패션잡화', 61), ('특허', 61), ('중고차', 61), ('단지정보', 61), ('시간대별', 61), ('러닝', 60), ('세탁기', 60), ('기업품목정보', 60), ('전용면적', 60), ('기사', 60), ('급상승키워드', 60), ('통합카테고리', 59), ('통합상품분류', 59), ('요일지역별', 59), ('맛집정보', 59), ('data', 58), ('지역시간대별', 58), ('데이터저널리즘', 58), ('맛집트렌드', 58), ('판매정보', 57), ('삼성카드', 57), ('기술', 57), ('페르소나', 57), ('온라인', 56), ('입주예정', 56), ('구매수량', 56), ('인기제품', 56), ('계절', 56), ('매칭정보', 55), ('SOHO', 55), ('요일별데이터', 55), ('순위', 54), ('전기차', 54), ('공급면적', 54), ('store', 54), ('complex', 54), ('지역경제', 54), ('코스피100대기업', 54), ('인기맛집', 53), ('구매', 52), ('패션아이템분석', 52), ('공기청정기', 52), ('제습기', 52), ('외식', 52), ('쿠콘', 52), ('교통', 51), ('라벨링', 51), ('분양', 51), ('신규아파트', 51), ('김치냉장고', 51), ('건조기', 51), ('ocb', 51), ('정보', 50), ('이미지', 50), ('전기밥솥', 50), ('구축', 50), ('무선청소기', 50), ('로봇청소기', 50), ('텔레비전', 50), ('일반세탁기', 50), ('"올시데이터', 49), ('드럼세탁기', 49), ('프로젝터', 49), ('유선청소기', 49), ('의류관리기', 49), ('경제감성', 49), ('사용자', 49), ('연평균', 48), ('View-T', 48), ('코로나19', 48), ('양문형냉장고', 48), ('리뷰추이', 48), ('경제뉴스', 48), ('금융뉴스', 48), ('뷰티속성', 48), ('서울', 47), ('30대', 47), ('식사시간', 47), ('주요미디어지수', 47), ('관심사', 47), ('쇼핑', 46), ('단지명', 46), ('카테고리별', 46), ('지역별강세', 46), ('맛집소개', 46), ('푸드매거진', 46), ('음식매거진', 46), ('직장인식권', 46), ('사진', 45), ('공동주택', 45), ('지역별매출수준', 45), ('포털', 45), ('포털검색', 45), ('20대', 45), ('총세대수', 45), ('건폐율', 45), ('코로나', 45), ('기획기사', 45), ('아티클', 45), ('조회통계', 45), ('온라인마켓', 44), ('용적율', 44), ('app', 44), ('식당사진', 44), ('잡지', 44), ('인터뷰', 44), ('핫플레이스지역', 44), ('휴무정보', 44), ('최고가격', 44), ('평균가격', 44), ('음식메뉴', 44), ('요식업', 44), ('조회수', 44), ('단지별면적', 43), ('면적정보', 43), ('기타공용면적', 43), ('기타공용상세', 43), ('주거공용상세', 43), ('통신사', 43), ('음식사진', 43), ('평가', 43), ('소득정보', 43), ('음식이름', 43), ('검색어', 43), ('차량', 42), ('선대별', 42), ('운송량', 42), ('사업자', 42), ('전문직비율', 42), ('소비정보', 42), ('2022년4월', 42), ('주거공용면적', 42), ('면적상세정보', 42), ('2021년12월', 42), ('2021년11월', 42), ('2022년3월', 42), ('2022년1월', 42), ('후기', 42), ('주가예측', 42), ('2022년2월', 42), ('big', 41), ('데이터루', 41), ('면적별정보', 41), ('일반맛집', 41), ('40대', 41), ('면적키정보', 41), ('식당명', 41), ('50대', 41), ('서비스', 41), ('교통량', 40), ('법인대표자비율', 40), ('산업', 40), ('서울직장인', 40), ('최고층', 40), ('소비패턴', 40), ('종목', 40), ('방문순위', 40), ('방문비율', 40), ('댓글', 40), ('메뉴명', 40), ('이슈', 39), ('이슈키워드', 39), ('실거래가매칭정보', 39), ('실거래가정보', 39), ('아파트매칭정보', 39), ('면적별매칭정보', 39), ('세분류전국', 39), ('종목별', 39), ('평균식대', 39), ('최고가', 39), ('부산시', 39), ('패션키워드랭킹', 39), ('단지정보매칭정보', 38), ('음식점명', 38), ('대구시', 38), ('아파트가격', 38), ('AI', 38), ('대전', 37), ('외제차보유비율', 37), ('금액', 37), ('POI', 37), ('세분류시군구', 37), ('엠코퍼레이션', 37), ('대안신용', 37), ('생산량', 37), ('에너지소비량', 37), ('패션위클리레포트', 37), ('윕스', 36), ('세분류', 36), ('자가거주비율', 36), ('아파트거주비율', 36), ('일반형냉장고', 36), ('마이데이터', 36), ('모니터링', 35), ('부산', 35), ('조선', 35), ('syrup', 35), ('음식이미지일련번호', 35), ('관심맛집', 35), ('가전', 35), ('상품', 34), ('경도', 34), ('해양', 34), ('육아', 34), ('음식트렌드', 34), ('다주택자비율', 34), ('입주예정아파트', 34), ('입주예정세대수', 34), ('폴리곤', 34), ('Marine', 33), ('Ship', 33), ('해사', 33), ('지역상권', 33), ('온누리스토어헬스케어', 33), ('구매정보', 33), ('방문데이터', 33), ('사업체', 33), ('위도', 32), ('Maritime', 32), ('Vessel', 32), ('2018', 32), ('60대이상', 32), ('미디어인덱스', 32), ('지하철역', 31), ('연령', 31), ('소비품목', 31), ('긍부정', 31), ('가공', 30), ('대구', 30), ('nice', 30), ('나이스디앤알', 30), ('출산', 30), ('맛집검색', 30), ('생활건강', 30), ('지역방문', 30), ('프롭테크', 30), ('자연어', 30), ('안전운항', 29), ('대중교통', 29), ('광주', 29), ('온라인판매내역온누리약국', 29), ('골프', 29), ('방송사', 29), ('패션키워드', 29), ('인구동향', 29), ('TV', 29), ('오픈마켓', 29), ('지역경계', 29), ('K패션', 29), ('인기', 29), ('통신', 28), ('GPS', 28), ('속도지표', 28), ('네비게이션', 28), ('활동성', 28), ('인기상권', 28), ('한류콘텐츠', 28), ('한류패션데이터', 28), ('지에스리테일', 28), ('Persona', 28), ('고객군', 28), ('고객세분화', 28), ('고객세그먼트', 28), ('명동', 28), ('항로정보', 27), ('톤마일', 27), ('선박별', 27), ('국가별', 27), ('2019년', 27), ('전기버스', 27), ('평균가', 27), ('매매가', 27), ('전세가', 27), ('전세율', 27), ('여의도', 27), ('KB차차차', 27), ('202111', 27), ('202110', 27), ('홍대', 27), ('합정', 27), ('202108', 27), ('대안신용정보', 27), ('건전성', 27), ('비재무정보', 27), ('부산연구원', 27), ('202107', 27), ('한류상품데이터', 27), ('의류매장', 27), ('커피전문점', 27), ('패션의류', 27), ('강남대로', 27), ('양식장', 26), ('지하철', 26), ('수도권', 26), ('Mobile', 26), ('차량이동', 26), ('거래데이터', 26), ('출산육아', 26), ('주력산업', 26), ('판매분석', 26), ('상권방문', 26), ('2025', 26), ('행태', 26), ('201910', 26), ('입주량', 26), ('공급량', 26), ('신규지역', 26), ('공급율', 26), ('농협카드', 26), ('데이터결합', 26), ('트렌드데이터', 26), ('202109', 26), ('coocon', 26), ('평균속도', 25), ('경제운항', 25), ('도로', 25), ('아이나비', 25), ('PND', 25), ('INAVI', 25), ('개인사업자', 25), ('모델', 25), ('202301', 25), ('데이터바우처', 25), ('KB캐피탈', 25), ('정치', 25), ('상권별방문', 25), ('인기상품', 25), ('신규부동산', 25), ('신규최고가', 25), ('이전최고가', 25), ('지역구분', 25), ('권역', 25), ('판매수량', 25), ('GS리테일', 25), ('디지털가전', 25), ('관광', 24), ('물류', 24), ('바다여행', 24), ('선주', 24), ('양식장경계', 24), ('상권분석', 24), ('온라인거래', 24), ('202106', 24), ('202201', 24), ('202206', 24), ('2019', 24), ('직장인야근', 24), ('야근', 24), ('석식', 24), ('석식이용데이터', 24), ('202102', 24), ('방문', 24), ('202210', 24), ('tv', 24), ('202103', 24), ('입주예정일', 24), ('202101', 24), ('202305', 24), ('식당이용', 24), ('거리별식당이용통계', 24), ('거리별데이터', 24), ('음식점운영현황', 24), ('음식점운영', 24), ('삼식별데이터', 24), ('202211', 24), ('신규기간', 24), ('한류패션', 24), ('중년', 24), ('202307', 24), ('결합', 24), ('202112', 24), ('거래량', 24), ('차차차', 24), ('202105', 24), ('202205', 24), ('요일별관심맛집', 24), ('요일별인기업종', 24), ('인기업종', 24), ('요일데이터', 24), ('202303', 24), ('202104', 24), ('202304', 24), ('202204', 24), ('실습선', 23), ('어촌관광', 23), ('여행', 23), ('유튜브', 23), ('인기음식', 23), ('직장인식사시간', 23), ('점심시간', 23), ('지역산업', 23), ('중고차순위', 23), ('맞춤형', 23), ('지표', 23), ('202208', 23), ('202203', 23), ('상권데이터', 23), ('202306', 23), ('분양아파트', 23), ('무료', 23), ('거리별', 23), ('가구인테리어', 23), ('운영', 23), ('202202', 23), ('면적별입주량', 23), ('기업종목', 23), ('202207', 23), ('202302', 23), ('화장품미용', 23), ('위치정보', 22), ('매매', 22), ('TAIMS', 22), ('따릉이', 22), ('교통데이터', 22), ('202009', 22), ('소호', 22), ('202008', 22), ('등록현황', 22), ('인기중고차', 22), ('연예', 22), ('LG', 22), ('전국상권', 22), ('202308', 22), ('부품', 22), ('202007', 22), ('시각화', 22), ('제품정보', 22), ('면적별가격', 22), ('사용인원비율', 22), ('업체', 22), ('부동산지인', 22), ('202011', 22), ('읍면동', 22), ('test', 22), ('co2', 21), ('선주별', 21), ('행정구역읍면동단위', 21), ('전세', 21), ('문화', 21), ('종합', 21), ('아파트시세', 21), ('지역평균가', 21), ('2016', 21), ('202012', 21), ('2017', 21), ('패션연관키워드', 21), ('상권방문데이터', 21), ('칼럼', 21), ('신차', 21), ('이동평균거래량', 21), ('브라우저', 21), ('202006', 21), ('트렌드레포트', 21), ('해외패션', 21), ('202010', 21), ('기업업종', 21), ('음성', 21), ('2015', 21), ('소비전국대장비율', 21), ('에너지', 21), ('음식가격', 21), ('품목', 21), ('디테일트렌드상의팬츠원피스', 21), ('진공청소기', 21), ('관측', 20), ('해양수산개발원', 20), ('평일', 20), ('비율', 20), ('시럽', 20), ('2014', 20), ('지역별시세', 20), ('지역별상권', 20), ('사설', 20), ('사설칼럼', 20), ('계약일기준', 20), ('신문', 20), ('구매순위', 20), ('수입', 19), ('행정구역 읍면동단위', 19), ('읍면동아파트시세', 19), ('사회', 19), ('청년', 19), ('패션연관키워드분석', 19), ('목적형쇼핑', 19), ('전국상권데이터', 19), ('예능', 19), ('10년이동평균', 19), ('인덱스', 19), ('해외패션상품분석', 19), ('Data', 19), ('검색', 19), ('성별데이터', 19), ('비중', 18), ('해상기상', 18), ('nox', 18), ('선종별', 18), ('펌프킨', 18), ('월별', 18), ('테스트', 18), ('인구', 18), ('보험', 18), ('맛집키워드', 18), ('삼성전자', 18), ('울산공실', 18), ('패션트렌드레포트', 18), ('인터넷쇼핑', 18), ('용품', 18), ('할부', 18), ('202005', 18), ('아파트월별거래량', 18), ('아파트거래비율', 18), ('지역별거래량', 18), ('계약일거래량', 18), ('지역별아파트최고가', 18), ('평형별최고가', 18), ('유입경로', 18), ('2013', 18), ('울산양산공급기업', 18), ('식품트렌드', 18), ('확진자', 18), ('유플러스', 18), ('해외패션상품정보', 18), ('사용자선호맛집', 18), ('수요공급', 18), ('메뉴가격', 18), ('수산물', 17), ('관광지', 17), ('컨테이너', 17), ('수출', 17), ('온실가스', 17), ('최대속도', 17), ('이산화탄소', 17), ('질소산화물', 17), ('버스', 17), ('인천', 17), ('신용', 17), ('맛집매거진', 17), ('맛집컨텐츠', 17), ('인기맛집트렌드', 17), ('맛집조회지역', 17), ('로그데이터', 17), ('사용자로그', 17), ('추석', 17), ('입주예정단지정보', 17), ('입주예정공급면적', 17), ('역내소비', 17), ('역외소비', 17), ('소비유입현황', 17), ('웰컴', 17), ('웰컴금융그룹', 17), ('아파트거래량', 17), ('지역별거래비율', 17), ('년도별거래량', 17), ('직업군', 17), ('201908', 17), ('패션상품분석', 17), ('Big', 17), ('STT', 17), ('TTS', 17), ('음성인식', 17), ('음성합성', 17), ('레이블링', 17), ('블러링', 17), ('메타', 17), ('전처리', 17), ('증권', 17), ('연령대별데이터', 17), ('나이별', 17), ('나이별인기맛집', 17), ('아파트입주예정', 17), ('면적별', 17), ('인물', 17), ('항만', 16), ('화물', 16), ('선박운항', 16), ('양식장메타데이터', 16), ('ton-mile', 16), ('포항테크노파크', 16), ('길', 16), ('결제', 16), ('영상', 16), ('세대수', 16), ('202001', 16), ('인기메뉴', 16), ('국가첨단전략', 16), ('바이오', 16), ('top10', 16), ('맛집조회데이터', 16), ('방문PV', 16), ('종합일간지', 16), ('인터넷신문', 16), ('삼성', 16), ('차트', 16), ('201912', 16), ('모바일트래픽', 16), ('아파트신고가', 16), ('면적별신고가', 16), ('이전실거래가최고가', 16), ('실거래가최고가갱신', 16), ('셀럽스타일', 16), ('한류스타', 16), ('모바일', 16), ('201907', 16), ('밀키트', 16), ('직업', 16), ('설날', 16), ('해외패션트렌드', 16), ('입주예정단지별상세정보', 16), ('지수정보', 16), ('지역지수', 16), ('특광도', 16), ('외식키워드', 16), ('키워드검색', 16), ('맛집검색트렌드', 16), ('url', 16), ('선호맛집', 16), ('가격분포', 16), ('가격대', 16), ('태그', 15), ('위성영상', 15), ('현황', 15), ('운송화물', 15), ('항로별', 15), ('울산', 15), ('개인', 15), ('요리', 15), ('계절별메뉴', 15), ('인기기사', 15), ('연체정보', 15), ('결제데이터', 15), ('일시불', 15), ('패션스타일', 15), ('201909', 15), ('국제', 15), ('201911', 15), ('청소년', 15), ('2012', 15), ('명절', 15), ('NICE', 15), ('NH농협은행', 15), ('발행기사수', 15), ('검색키워드', 15), ('선물세트', 15), ('우체국', 15), ('환경', 14), ('AIS', 14), ('운항거리', 14), ('동영상', 14), ('NH트렌드', 14), ('재무정보', 14), ('직장인식사', 14), ('소비재', 14)]

In [ ]:
names = []
for name, count in tags:
    names.append(name)

In [ ]:
names

In [ ]:
# 계층 분류 구조
category_tree = {
  "소비/유통": {
    "상품": ["유통", "판매", "쇼핑", "구매", "온라인몰", "물류"],
    "소비행태": ["소비", "소비트렌드", "소비자", "마케팅"],
    "거래정보": ["구매후기", "리뷰", "리뷰정보"]
  },
  "공간/위치": {
    "인구이동": ["유동인구", "방문인구", "직장인구", "서비스인구", "주거인구"],
    "위치기반": ["위치데이터", "위치정보", "입출항", "스케줄"],
    "관광": ["관광", "관광지", "여행"]
  },
  "데이터분석": {
    "빅데이터": ["빅데이터", "데이터", "분석", "AI", "텍스트마이닝"],
    "미디어분석": ["SNS", "소셜", "블로그", "트위터", "인스타그램", "미디어"],
    "자연어처리": ["감성분석", "의견분석", "텍스트"]
  },
  "산업/기술": {
    "IT기술": ["IoT", "클라우드", "AI", "인공지능", "플랫폼"],
    "해양/수산": ["어업", "양식", "해양", "어선정보", "조업", "실습선", "어선", "여객선", "선박"],
    "에너지": ["풍력", "태양광", "신재생에너지"]
  },
  "공공/행정": {
    "정책": ["정부", "행정", "공공데이터"],
    "통계": ["통계청", "자료", "데이터개방"],
    "지역": ["광역시", "시군구", "지역"]
  }
}

# 소분류 -> (대분류, 중분류) 맵핑
subcategory_map = {}
for big, mids in category_tree.items():
    for mid, subs in mids.items():
        for sub in subs:
            subcategory_map[sub] = (big, mid, sub)


In [ ]:
subcategory_map

In [ ]:
subcategory_map.keys()

In [ ]:
import pandas as pd

# 결과 열 초기화
df['대분류'] = None
df['중분류'] = None
df['소분류'] = None

missings = {}

count = 0
for idx, row in df.iterrows():
    tags = row['tags_list']
    found = False
    for tag in tags:
        for subcategory, category in subcategory_map.items():
            if subcategory in tag:
                big, mid, sub = category
                df.at[idx, '대분류'] = big
                df.at[idx, '중분류'] = mid
                df.at[idx, '소분류'] = sub
                found = True            
                break  # 하나라도 매칭되면 종료
    if found == False:
        print(count, tags)
        count += 1

In [ ]:
missings